We can expand $\hat{H}_{dse}$ in terms of the dipole operator (with electronic and nuclear contributions) and dipole expectation values as follows:
\begin{align}
    \hat{H}_{dse} &= \frac{1}{2} \sum_{\xi, \xi'} \sum_{i \neq j} \lambda^{\xi} \lambda^{\xi'} \mu^{\xi}(x_i) \mu^{\xi'}(x_j) \\
    &+  \frac{1}{2} \sum_{\xi, \xi'} \sum_i \frac{1}{2} \lambda^{\xi} \lambda^{\xi'} Q^{\xi \xi'}(x_i) \\ 
   & + \left(\lambda \cdot \mu_{nuc} - \lambda \cdot \langle \mu \rangle \right) \sum_{\xi} \sum_i \lambda^{\xi} \mu^{\xi} (x_i)  \\
  &+ \frac{1}{2} \left( \lambda \cdot \mu_{nuc} \right)^2  - \left( \lambda \cdot \langle \mu \rangle \right) \left( \lambda \cdot \mu_{nuc} \right) + \frac{1}{2} \left( \lambda \cdot \langle \mu \rangle \right)^2
\end{align} 

In the above expansion of $\hat{H}_{dse}$ we have specifically indicated that the product of electronic dipole operators contains 2-electron contributions when $i \neq j$,
and 1-electron quadrupole contributions when $i = j$.  Furthermore, a
one-electron term arises that contains the electronic dipole operator
scaled by $\lambda \cdot \mu_{nuc}$.  In the QED-RHF procedure, the additional one-electron terms above will be added to $H_{core}$ and the additional two-electron terms above will be included in the density-matrix dependent terms in the Fock operator:
\begin{equation}
    F_{\mu \nu} = H_{\mu \nu} + G_{\mu \nu}
\end{equation}
where
\begin{align}
    H_{\mu \nu} &= h_{\mu \nu} - \frac{1}{2}\sum_{\xi, \xi'} \lambda^{\xi} \lambda^{\xi'} Q^{\xi \xi'}_{\mu \nu} \\
    &+ \left(\lambda \cdot \mu_{nuc} - \lambda \cdot \langle \mu \rangle \right) \sum_{xi} \lambda^{\xi} \mu^{\xi}_{\mu \nu}
\end{align}
and
\begin{align}
  G_{\mu \nu} &=   
  \left( 2(\mu\,\nu\left|\,\lambda\,\sigma) - (\mu\,\lambda\,\right|\nu\,\sigma) \right) D_{\lambda\sigma} \\
  & + \left( \sum_{\xi \xi'} \lambda^{\xi} \lambda^{\xi'} \left(\mu^{\xi}_{\mu \nu} \mu^{\xi'}_{\lambda \sigma} - \frac{1}{2} \mu^{\xi}_{\mu \lambda} \mu^{\xi'}_{\nu \sigma} \right)
  \right)D_{\lambda\sigma},
\end{align}
leading to the total QED-RHF energy being
\begin{equation}
    E_{QED-RHF} =  (F_{\mu\nu} + H_{\mu\nu})D_{\mu\nu} + E_{nuc} + d_c
\end{equation}
where $d_c =\frac{1}{2} \left( \lambda \cdot \mu_{nuc} \right)^2  - \left( \lambda \cdot \langle \mu \rangle \right) \left( \lambda \cdot \mu_{nuc} \right) + \frac{1}{2} \left( \lambda \cdot \langle \mu \rangle \right)^2$
Note: In the above, the quadrupole terms have a negative sign!  This is because we are really using the quadrupole integrals in place of products of dipole integrals, but these differ by a factor of the electron charge which is negative 1 in atomic units.  For example:
\begin{equation}
\hat{Q}_{el}^{xx} \propto q_{el} \hat{x} \hat{x}
\end{equation}
whereas 

\begin{equation}
\hat{\mu}_{el}^x \hat{\mu}_{el}^x \propto q_{el} q_{el} \hat{x} \hat{x}
\end{equation}

Hence, we use the quadrupole integrals but multiply by -1 to compensate!


This notebook uses the CQED-RHF function written within `helper_cqed_rhf.py` and runs
it on a case for which the result is known from Table I and II results [here](https://arxiv.org/pdf/2011.12768.pdf).
There is an assert statement that tests that the energy between our code and the known answer agrees to within 5e-5 Hartrees, which is the error due to the fact that a density fitting approximation was used in the Eugene's paper which we do not employ here!

In [ ]:
from __future__ import print_function

"""
A reference implementation of cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la
import time
from helper_cqed_rhf import *

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
#psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

# basis set etc
psi4.set_options({'basis':        'def2-tzvppd',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10})

NaF_string = """

0 1
    NA           0.000000000000     0.000000000000    -0.875819904077
    F            0.000000000000     0.000000000000     1.059820520433
no_reorient
#nocom
symmetry c1
"""

NaCl_string = """

0 1
    NA           0.000000000000     0.000000000000    -1.429419641344
    CL           0.000000000000     0.000000000000     0.939751385626
no_reorient
#nocom
symmetry c1
"""

expected_NaF =  -261.371070718358
expected_NaCl = -621.438985539266

# electric field
Ex = 0.
Ey = 0.
Ez = 0.01

lam = np.array([Ex, Ey, Ez])




In [ ]:
# run cqed_rhf on NaF and compare to expected answer
cqed_rhf_dict = cqed_rhf(lam, NaF_string)
em_cqed_rhf_e = cqed_rhf_dict['cqed_rhf_energy']
em_rhf_e = cqed_rhf_dict['rhf_energy']
assert np.isclose(em_cqed_rhf_e, expected_NaF,5e-5)
print("def2-tzvppd RHF energy of NaF:               ", em_rhf_e)
print("def2-tzvppd CQED-RHF energy of NaF:          ", em_cqed_rhf_e)
print("reference def2-tzvppd CQED-RHF energy of NaF:", expected_NaF)

cqed_rhf_dict = cqed_rhf(lam, NaCl_string)
em_rhf_e = cqed_rhf_dict['rhf_energy']
em_cqed_rhf_e = cqed_rhf_dict['cqed_rhf_energy']
assert np.isclose(em_cqed_rhf_e, expected_NaCl,5e-5)
print("def2-tzvppd RHF energy of NaCl:               ", em_rhf_e)
print("def2-tzvppd CQED-RHF energy of NaCl:          ", em_cqed_rhf_e)
print("reference def2-tzvppd CQED-RHF energy of NaCl:", expected_NaCl)


In [ ]:
# change to the cc-pVDZ basis set
psi4.set_options({'basis':        'cc-pVDZ'})


In [ ]:
# template for the z-matrix for MgH+
mol_tmpl = """Mg
H 1 **R**
symmetry c1
1 1"""

In [ ]:
# array of bondlengths for MgH+
r_array = np.array([1.0, 1.1, 1.2, 1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3])

# lambda arrays
l_big = np.array([0.0, 0.0, 0.1])
l_med = np.array([0.0, 0.0, 0.01])

# empty arrays for the energies 
# for "big lambda" -> (0.1, 0.1, 0.1)
cqed_energy_array_l_big = np.zeros_like(r_array)
# for "medium lambda" -> (0.01, 0.01, 0.01)
cqed_energy_array_l_med = np.zeros_like(r_array)
rhf_energy_array = np.zeros_like(r_array)


# loop over the different bond-lengths, create different instances
# of HF molecule
ctr = 0
for r in r_array:
    molstr = mol_tmpl.replace("**R**", str(r))
    print(molstr)
    med_dict = cqed_rhf(l_med, molstr)
    big_dict = cqed_rhf(l_big, molstr)
    rhf_energy_array[ctr] = med_dict['rhf_energy']
    cqed_energy_array_l_med[ctr] = med_dict['cqed_rhf_energy']
    cqed_energy_array_l_big[ctr] = big_dict['cqed_rhf_energy']
    ctr+=1
    

In [ ]:
from matplotlib import pyplot as plt
plt.plot(r_array, rhf_energy_array, 'r-o', label="$\lambda$=(0,0,0) a.u.")
plt.plot(r_array, cqed_energy_array_l_med, 'b-*', label="$\lambda$=(0, 0, 0.01) a.u.")
plt.plot(r_array, cqed_energy_array_l_big, 'p--', label="$\lambda$=(0, 0, 0.1) a.u.")
plt.xlabel("Bondlength (Angstroms)")
plt.ylabel("Energy (Hartrees)")
plt.legend()
plt.show()